# Minesweeper Puzzle

The player is presented with a grid of squares. Some randomly selected squares, unknown to the player, contain mines. Other squares may display numbers indicating the count of mines present in the immediate neighborhood of the squares. Find all mines. 

See https://de.wikipedia.org/wiki/Minesweeper or go to http://minesweeperonline.com/ to play

Imports

In [1]:
from ortools.constraint_solver import pywrapcp
from itertools import product
import numpy as np

Pretty-print square minesweeper board

In [2]:
def pretty_print(mines, labels):
    for i in range(len(labels)):
        for j in range(len(labels)):
            if labels[i][j] != "":
                print("[ {}]\t".format(labels[i][j]), end='')
            elif mines[i][j].Value() == 1:
                print("[💣]\t", end='')
            else:
                print("[  ]\t", end='')
        print("\n")
    print("\n\n")

Some Minesweeper examples, the first one is from the lecture

In [3]:
mine_field1 = [
    ["2", "", "", "", "1"],
    ["2", "", "4", "3", ""],
    ["", "2", "", "1", ""],
    ["", "1", "", "3", ""],
    ["1", "", "", "", ""],
]
mine_field2 = [
    ["1", "", "", "", "2", "", "", "1"],
    ["", "1", "2", "", "3", "", "", "1"],
    ["", "2", "", "1", "", "", "", "0"],
    ["", "2", "1", "", "", "2", "3", "1"],
    ["", "", "", "2", "", "", "", ""],
    ["1", "", "", "", "4", "3", "", ""],
    ["", "1", "", "", "4", "", "3", ""],
    ["1", "", "", "2", "", "2", "", "1"],
]
mine_field3 = [
    ["", "", "", "", "", "1", "", ""],
    ["1", "2", "2", "1", "", "", "1", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "0", "", "", "1", "", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "1", "", "2", "3", "", ""],
    ["", "", "", "", "", "", "2", "1"],
    ["", "1", "", "2", "2", "", "2", ""],
]
mine_field4 = [
    ["1", "", "", "2", "", "2", "", "2", "", ""],
    ["", "3", "2", "", "", "", "4", "", "", "1"],
    ["", "", "", "1", "3", "", "", "", "4", ""],
    ["3", "", "1", "", "", "", "3", "", "", ""],
    ["", "2", "1", "", "1", "", "", "3", "", "2"],
    ["", "3", "", "2", "", "", "2", "", "1", ""],
    ["2", "", "", "3", "2", "", "", "2", "", ""],
    ["", "3", "", "", "", "3", "2", "", "", "3"],
    ["", "", "3", "", "3", "3", "", "", "", ""],
    ["", "2", "", "2", "", "", "", "", "2", ""],
]

Pick one of the examples

In [4]:
game = mine_field1

Create constraint solver

In [5]:
solver = pywrapcp.Solver("Minesweeper")

In [6]:
# calculate the board size
n, m = len(game), len(game[0])

# create n*n board for the solution
board = [[None for _j in range(m)] for _i in range(n)]

# assign pre-defined values
for i in range(n):
    for j in range(m):
        if game[i][j] == "":
            # we need to either mark it as mine or non-mine
            board[i][j] = solver.IntVar(0, 1)
        else:
            # field is a pre-defined constant
            board[i][j] = solver.IntVar(0, 8)
            solver.Add(board[i][j] == int(game[i][j]))

            
def get_neighbors(board_size, cell):
    I, J = board_size
    i, j = cell
    neighbors = []
    for x in range(i - 1, i + 2):
        for y in range(j - 1, j + 2):
            if -1 < i <= I and -1 < j <= J and (i != x or j != y) and 0 <= x < I and 0 <= y < J:
                neighbors.append((x, y))
    return neighbors


# Constraint: amount of mines as neighbours
for i in range(n):
    for j in range(m):
        # only check pre-defined fields with actual number in it
        if game[i][j] == "":
            continue
            
        # get all neighbors
        neighbors = get_neighbors((n, m), (i, j))
        # remove pre-defined fields from neighbors
        neighbors = [(x, y) for x, y in neighbors if game[x][y] == ""]
        # value of a pre-defined field has to be equal the same of it's valid neighbors.
        # whereas a valid neighbor either contains a 0 for no-mine or a 1 for a mine.
        solver.Add(solver.Sum([board[k][l] for k, l in neighbors]) == board[i][j])

Configure solver

In [7]:
# Replace this by a list of decision varianles in your model
all_vars = list(np.concatenate(board))

db = solver.Phase(all_vars, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [8]:
solver.NewSearch(db)
while solver.NextSolution():
    pretty_print(board, game)

[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	






Cleanup

In [9]:
solver.EndSearch()

Print solver information

In [10]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 1
Runtime:   55ms
Failures:  2
Branches:  4 
